In [1]:
import os
import pandas as pd
import  matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from transformers import AutoTokenizer, AutoModel
from transformers import DefaultDataCollator
from datasets import load_dataset


from OneCrossInitialParameter1 import InitialParameter

import sys
sys.path.append('./mymodelkit/')
from mymodelkit import TextDataAugmentation, jumanTextAlignment

In [2]:
parameter = InitialParameter()
parameter

InitialParameter(implemnt_len=1, train_data_dir='./data/train', train_list=['cross_noaug_suisho_train.xlsx', 'cross_noaug_yokusei_train.xlsx', 'cross_noaug_hagemashi_train.xlsx', 'cross_noaug_ganbou_train.xlsx'], test_data_dir='./data/test', test_list=['cross_noaug_suisho_test.xlsx', 'cross_noaug_yokusei_test.xlsx', 'cross_noaug_hagemashi_test.xlsx', 'cross_noaug_ganbou_test.xlsx'], save_dir='/home/wakasugi/model_create/MultiFacilitatedAxis/result', save_list=['cross_noaug_one_suisho', 'cross_noaug_one_yokusei', 'cross_noaug_one_hagemashi', 'cross_noaug_one_ganbou'], metrics_data_result='one_result_noaugcrross.xlsx', metrics_model_result='one_model_noaugcrross.xlsx', use_model_list=['nlp-waseda/roberta-base-japanese', 'nlp-waseda/roberta-base-japanese', 'nlp-waseda/roberta-base-japanese', 'nlp-waseda/roberta-base-japanese'], text_column_name='ツイート', label_column_name='判定結果', label_list=['推奨', '行動抑制', '励まし', '願望'], JaToEn_label={'推奨': 'recommend', '行動抑制': 'suppression', '励まし': 'encourag

In [3]:
ReadFile = './original/sokushinラベル確定_result.xlsx'

all_data_pd = pd.read_excel(ReadFile)
all_data_pd = all_data_pd[all_data_pd['判定結果'] != 'その他']
all_data_pd.reset_index(drop=True,inplace=True)
all_data_pd

,ツイート,判定結果
0,おはようございます週末関東も台風の影響があるようです。最近、雨や風での被害が多いのに台風がま...,推奨
1,【台風被害に伴う施設開放終了のお知らせ】10月15日をもちまして、施設の開放を終了しました。...,推奨
2,ありがとうございます飲料水や食料品、電池などの備蓄は南海トラフに備えてしてあったのですが、断...,推奨
3,/ 台風14号 九州を北上 朝には中国・四国地方に接近する見込み 土砂災害などに厳重警戒 \,推奨
4,本日は 『オレンチェ2マンライブ』 静岡UHUにて無事開催します! 台風の影響で予定し...,推奨
...,...,...
9604,おはようございます台風が、また来ますね天気の悪い日には、美術館博物館へ。陶磁美術館へようこそ...,推奨
9605,ありがとうございます!私や家族は大丈夫ですが近所の方や同じ区内で困ってる方もいると思います...,願望
9606,09/07 05:08 発表 [令和元年台風第13号に関する奄美地方気象情報] 奄美地方では...,推奨
9607,1/2【北海道地方 気象情報 2019年10月12日 16:49】北海道地方では、13日は...,推奨


In [4]:
#データのラベル構成確認
from collections import defaultdict
result_dict = defaultdict(int)

for label in parameter.label_list:
    result_dict[label] = 0
    
for idx, row in all_data_pd.iterrows():
    label_ = row[1]
    #if '|' in label_:
    #    result_dict[label_] += 1 #ラベルのタプルを数えるため
    #label_ = label_.split('|')
    #for la in label_:
    #    result_dict[la] += 1
    result_dict[label_] += 1
    
result_dict

defaultdict(int,
            {'推奨': 8078,
             '行動抑制': 122,
             '励まし': 230,
             '願望': 617,
             '推奨|励まし': 249,
             '推奨|行動抑制': 132,
             '推奨|願望': 163,
             '願望|励まし': 10,
             '行動抑制|願望': 6,
             '推奨|願望|励まし': 2})

In [15]:
for label in parameter.label_list:
    all_data_pd[label] = 0

for idx, row in all_data_pd.iterrows():
    labels = row[1].split('|')
    for label in labels:
        all_data_pd.loc[idx, label] = 1
all_data_pd.drop('判定結果', axis=1, inplace=True)
all_data_pd.iloc[50:100, :]

,ツイート,推奨,行動抑制,励まし,願望
50,大型で強い台風14号は、19日9時現在、北九州市付近を時速約15kmの速さで北北東へ進んで...,1,0,0,0
51,台風に気をつけてくださいね,1,0,0,0
52,おはぎを自分で献立に入れてたのに忘れてました… 台風直撃?! 気をつけてね,1,0,0,0
53,台風15号で被災された皆様へ八街(やちまた)少年院において被災された方々への支援活動(給水・...,1,0,0,0
54,今日も楽しく出来たなら良かったねぇ?…おつかれちゃーん(≧∀≦)ノ…。アイス食べたのかぁ?…...,1,0,0,0
55,おはようございます中国地方が暴風域9月23日(月)台風17号 は島根県沖の日本海を北東に進ん...,1,0,0,0
56,令和元年台風15号被害支援金募金受付のお知らせ|ファミリーマート#復興に向けて手を繋ごう#台...,1,0,1,0
57,こんばんわ お疲れ様でした 台風が又 やって来そうですね。 気温も上がったり下がったり 着い...,1,0,0,0
58,TBS NEWS 防災さんはTwitterを使っています: 「台風は事前に準備ができる災害で...,1,0,0,0
59,はいですー(´゜д゜`)今度は台風にも注意しましょうです!!,1,0,0,0


In [16]:
metrics_dict = defaultdict(list)
all_train = pd.DataFrame()
all_test = pd.DataFrame()
crross_num = parameter.training_args.num_train_epochs
for label, trafile,tesfile in zip(parameter.label_list[0:], parameter.train_list[0:], parameter.test_list[0:]): 
    print(label)
    #対象のデータ取得
    correct_pd = all_data_pd[ (all_data_pd[label] == 1)]
    no_correct_pd = all_data_pd[ (all_data_pd[label] == 0)]
    
    #correct_pdとno_correct_pdの行数の最小値を取得する（アンダーサンプリングのために）
    min_value = min(len(correct_pd), len(no_correct_pd))
    
    #アンダーサンプリング
    correct_pd = correct_pd.sample(min_value)
    no_correct_pd = no_correct_pd.sample(min_value)
    
    correct_pd['implimention_num'] = -1
    no_correct_pd['implimention_num'] = -1
    
    #交差検証データ作成
    kf = KFold(n_splits=crross_num, shuffle=True, random_state=1)
    idx = 0
    for train_list, test_list in kf.split(correct_pd):
        #交差検証用の番号を割り当てる
        correct_pd.iloc[test_list, list(correct_pd.columns).index("implimention_num")] = idx
        no_correct_pd.iloc[test_list, list(correct_pd.columns).index("implimention_num")] = idx
        idx += 1
    #学習データとテストデータの結合
    tmp_train = pd.concat([correct_pd, no_correct_pd], axis=0, ignore_index=True)
    tmp_test = pd.concat([correct_pd, no_correct_pd], axis=0, ignore_index=True)
    
    print('train_len:', len(tmp_train))
    print('test_len:', len(tmp_test))

    #学習データ全てをテキスト拡張の対象とする
    #textaugmentation = TextDataAugmentation.TextAugmentation(parameter)
    #train_aug = textaugmentation.run(tmp_train)
    train_aug = tmp_train

    #学習データとテストデータの分かち書きを行う
    jumantextalignment = jumanTextAlignment.jumanTextAlignment(parameter)
    print(label)
    print("学習データの分かち書き開始")
    juman_augmented_pd = jumantextalignment.run(train_aug)
    print("テストデータの分かち書き開始")
    test_juman_augmented_pd = jumantextalignment.run(tmp_test)
    
    metrics_dict['label'].append(label)
    metrics_dict['correct_pd'].append(len(correct_pd))
    metrics_dict['no_correct_pd'].append(len(no_correct_pd))
    metrics_dict['tmp_train'].append(len(tmp_train))
    metrics_dict['tmp_test'].append(len(tmp_test))
    metrics_dict['train_correct'].append(len(tmp_train[(tmp_train[label]==1) & (tmp_train["implimention_num"]!=0)]))
    metrics_dict['train_no_correct'].append(len(tmp_train[(tmp_train[label]==0) & (tmp_train["implimention_num"]!=0)]))
    metrics_dict['test_correct'].append(len(tmp_test[(tmp_test[label]==1) & (tmp_test["implimention_num"]==0)]))
    metrics_dict['test_no_correct'].append(len(tmp_test[(tmp_test[label]==0)& (tmp_test["implimention_num"]==0)]))
    metrics_dict['train_correct_augmentation'].append(len(train_aug[(train_aug[label]==1) & (train_aug["implimention_num"]!=0)]))
    metrics_dict['train_no_correct_augmentation'].append(len(train_aug[(train_aug[label]==0) & (train_aug["implimention_num"]!=0)]))
    

    juman_augmented_pd.to_excel(os.path.join(parameter.train_data_dir, trafile), index=None)
    test_juman_augmented_pd.to_excel(os.path.join(parameter.test_data_dir, tesfile), index=None)
    
    #全データ結合
    all_train = pd.concat([all_train, tmp_train], axis=0, ignore_index=True)
    all_test = pd.concat([all_test, tmp_test], axis=0, ignore_index=True)

推奨
train_len: 1970
test_len: 1970
推奨
学習データの分かち書き開始
テストデータの分かち書き開始
行動抑制
train_len: 520
test_len: 520
行動抑制
学習データの分かち書き開始
テストデータの分かち書き開始
励まし
train_len: 982
test_len: 982
励まし
学習データの分かち書き開始
テストデータの分かち書き開始
願望
train_len: 1596
test_len: 1596
願望
学習データの分かち書き開始
テストデータの分かち書き開始


In [17]:
metrics_dict

defaultdict(list,
            {'label': ['推奨', '行動抑制', '励まし', '願望'],
             'correct_pd': [985, 260, 491, 798],
             'no_correct_pd': [985, 260, 491, 798],
             'tmp_train': [1970, 520, 982, 1596],
             'tmp_test': [1970, 520, 982, 1596],
             'train_correct': [788, 208, 392, 638],
             'train_no_correct': [788, 208, 392, 638],
             'test_correct': [197, 52, 99, 160],
             'test_no_correct': [197, 52, 99, 160],
             'train_correct_augmentation': [788, 208, 392, 638],
             'train_no_correct_augmentation': [788, 208, 392, 638]})

In [18]:
pd_ = pd.DataFrame.from_dict(metrics_dict)
pd_

,label,correct_pd,no_correct_pd,tmp_train,tmp_test,train_correct,train_no_correct,test_correct,test_no_correct,train_correct_augmentation,train_no_correct_augmentation
0,推奨,985,985,1970,1970,788,788,197,197,788,788
1,行動抑制,260,260,520,520,208,208,52,52,208,208
2,励まし,491,491,982,982,392,392,99,99,392,392
3,願望,798,798,1596,1596,638,638,160,160,638,638


In [19]:
pd_.to_excel(os.path.join(parameter.save_dir, parameter.metrics_data_result), index=None)

In [20]:
train_aug[(train_aug[label]==1) & (tmp_test["implimention_num"]==0)].index

Int64Index([  3,   8,  16,  19,  23,  35,  41,  47,  49,  50,
            ...
            727, 738, 739, 761, 764, 773, 777, 779, 782, 785],
           dtype='int64', length=160)

In [21]:
train_aug[(train_aug[label]==1) & (tmp_test["implimention_num"]==0)]

,ツイート,推奨,行動抑制,励まし,願望,implimention_num
3,大網、まったく進歩が感じられません。停電8日目。もう本当に絶望すら感じるんだけど…テレビ等で...,0,0,0,1,0
8,もみじ饅頭美味しい、、 皆さま台風の影響は大丈夫ですか、? 愛媛は大丈夫そうです!! 今...,0,0,0,1,0
16,まず先日の「台風14号に関する災害対策室」の活動報告をして下さい どうせ何もしてないだろう...,0,0,0,1,0
19,大阪タワレコカフェ譲っていただいた方を探しています。 台風の影響により休業となり払い戻し対象...,0,0,0,1,0
23,皆さまおはようございます 本日は台風の影響で ランチ営業のみの営業予定です。 お客様にはご...,0,0,0,1,0
...,...,...,...,...,...,...
773,千葉県、台風の影響がひどいです。あたしんちは窓が割れ、瓦が飛び、70後半のおじいちゃん80代...,0,0,0,1,0
777,【必要としている方に届けてください】台風19号で被害を受けた地域では、仮設住宅の申し込みや建...,1,0,0,1,0
779,台風が相手だから、何とも言えないですけどね。各自、情報確認していただきながら安全に動いていた...,1,0,0,1,0
782,【NHKニュース・防災アプリ】 大雨で自宅近くの川の様子が気になる。 両親に避難情報をいち...,0,0,0,1,0


In [12]:
all_train.to_excel(f'./data/tmp_traindata{parameter.metrics_data_result}', index=None)
all_test.to_excel(f'./data/tmp_testdata{parameter.metrics_data_result}', index=None)

# データ結合

In [13]:
all_train = pd.read_excel('tmp_train.xlsx')
all_test = pd.read_excel('tmp_test.xlsx')

In [14]:
all_train

,ツイート,推奨,行動抑制,励まし,願望,implimention_num
0,おはようございます(*´`*)ﾉ 台風がぁぁ( ° 安全第一で今日もまいりましょう(**),1,0,0,0,1
1,台風きたら休めよ,1,0,0,0,3
2,20190923(天気)秋分の日の祝日、2019年9月23日(月)6時現在、美作大学のある津...,1,0,0,0,0
3,FXの利益の半分プレゼント中です 今回はなんと5名に65万当たるチャンス! 本日19時締切...,1,0,0,0,0
4,台風15号ファクサイにご注意ください。#北習志野 #お天気 [北習志野店],1,0,0,0,2
...,...,...,...,...,...,...
5063,お疲れ様でした 台風の影響が出て大変そうでしたが無事LIVE出来たみたいで良かった 明日は更...,1,0,0,0,4
5064,台風15号の影響で千葉県市原市付近は信号は機能せず、コンビニは営業停止。1.2件の話ではあ...,1,0,0,0,3
5065,おはちぇり 今日は祝日だけど、外仕事です 制作中のソフトウェアも、リリース開始まで後半年 ...,0,0,1,0,1
5066,まあお仕事でしたか! それはお疲れさまです♀ 今読み返したら、レオンと呼び捨てになっていまし...,1,0,0,0,3


In [15]:
from MultiCrossInitialParameter1 import InitialParameter

parameter = InitialParameter()
parameter

InitialParameter(implemnt_len=1, train_data_dir='./data/train', train_list=['all_noaug_train.xlsx'], test_data_dir='./data/test', test_list=['all_noaug_test.xlsx'], save_dir='/home/wakasugi/model_create/MultiFacilitatedAxis/result', save_list=['multi_noaug_sokushin'], metrics_data_result='multi_result_noaugcrross', metrics_model_result='multi_model_noaugcrross', use_model_list=['nlp-waseda/roberta-base-japanese'], text_column_name='ツイート', label_column_name='判定結果', label_list=['推奨', '行動抑制', '励まし', '願望'], JaToEn_label={'推奨': 'recommend', '行動抑制': 'suppression', '励まし': 'encouragement', '願望': 'desire'}, JaToEn_column={'ツイート': 'tweet', '判定結果': 'label'}, max_length=256, training_args=TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=False,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=No

In [25]:
#データ拡張する前の学習データとテストデータを取得
duplicated_train = all_train.copy()
duplicated_train['implimention_num'] = -1
duplicated_test = all_test.copy()
duplicated_test['implimention_num'] = -1

#データの重複削除
duplicated_train = duplicated_train[[not target for target in duplicated_train.duplicated()]]
duplicated_train = duplicated_train.reset_index(drop=True)
duplicated_test = duplicated_test[[not target for target in duplicated_test.duplicated()]]
duplicated_test = duplicated_test.reset_index(drop=True)

In [26]:
duplicated_train

,ツイート,推奨,行動抑制,励まし,願望,implimention_num
0,おはようございます(*´`*)ﾉ 台風がぁぁ( ° 安全第一で今日もまいりましょう(**),1,0,0,0,-1
1,台風きたら休めよ,1,0,0,0,-1
2,20190923(天気)秋分の日の祝日、2019年9月23日(月)6時現在、美作大学のある津...,1,0,0,0,-1
3,FXの利益の半分プレゼント中です 今回はなんと5名に65万当たるチャンス! 本日19時締切...,1,0,0,0,-1
4,台風15号ファクサイにご注意ください。#北習志野 #お天気 [北習志野店],1,0,0,0,-1
...,...,...,...,...,...,...
3606,#鋸南町 #災害ボランティア 受付中止悪天候が予想されるため、#ボランティア 受付中止です...,1,0,0,0,-1
3607,お疲れ様でした 台風の影響が出て大変そうでしたが無事LIVE出来たみたいで良かった 明日は更...,1,0,0,0,-1
3608,台風15号の影響で千葉県市原市付近は信号は機能せず、コンビニは営業停止。1.2件の話ではあ...,1,0,0,0,-1
3609,まあお仕事でしたか! それはお疲れさまです♀ 今読み返したら、レオンと呼び捨てになっていまし...,1,0,0,0,-1


In [27]:
duplicated_test

,ツイート,推奨,行動抑制,励まし,願望,implimention_num
0,おはようございます(*´`*)ﾉ 台風がぁぁ( ° 安全第一で今日もまいりましょう(**),1,0,0,0,-1
1,台風きたら休めよ,1,0,0,0,-1
2,20190923(天気)秋分の日の祝日、2019年9月23日(月)6時現在、美作大学のある津...,1,0,0,0,-1
3,FXの利益の半分プレゼント中です 今回はなんと5名に65万当たるチャンス! 本日19時締切...,1,0,0,0,-1
4,台風15号ファクサイにご注意ください。#北習志野 #お天気 [北習志野店],1,0,0,0,-1
...,...,...,...,...,...,...
5059,#鋸南町 #災害ボランティア 受付中止悪天候が予想されるため、#ボランティア 受付中止です...,1,0,0,0,-1
5063,お疲れ様でした 台風の影響が出て大変そうでしたが無事LIVE出来たみたいで良かった 明日は更...,1,0,0,0,-1
5064,台風15号の影響で千葉県市原市付近は信号は機能せず、コンビニは営業停止。1.2件の話ではあ...,1,0,0,0,-1
5066,まあお仕事でしたか! それはお疲れさまです♀ 今読み返したら、レオンと呼び捨てになっていまし...,1,0,0,0,-1


In [29]:
#データのラベル構成確認
from collections import defaultdict
result_dict = defaultdict(int)

for label in parameter.label_list:
    result_dict[label] = 0
    
for idx, row in duplicated_train.iterrows():
    for label_ in  parameter.label_list:
        if row[label_] == 1:
            result_dict[label_] += 1
    
result_dict

defaultdict(int, {'推奨': 2626, '行動抑制': 260, '励まし': 491, '願望': 798})

In [26]:
metrics_dict = defaultdict(list)
crross_num=5
for trafile,tesfile in zip(parameter.train_list[0:], parameter.test_list[0:]): 
    
    #交差検証データ作成
    kf = KFold(n_splits=crross_num, shuffle=True, random_state=1)
    idx = 0
    for train_list, test_list in kf.split(duplicated_train):
        #交差検証用の番号を割り当てる
        duplicated_train.iloc[test_list, list(duplicated_train.columns).index("implimention_num")] = idx
        duplicated_test.iloc[test_list, list(duplicated_test.columns).index("implimention_num")] = idx
        idx += 1
    
    print('train_len:', len(duplicated_train))
    print('test_len:', len(duplicated_test))

    #学習データ全てをテキスト拡張の対象とする
    textaugmentation = TextDataAugmentation.TextAugmentation(parameter)
    #train_aug = textaugmentation.run(duplicated_train)
    #kwarg = {"alpha_sr":0.05, "alpha_ri":0.05, "alpha_rs":0.05, "p_rd":0.05, "num_aug":8}
    #train_aug = textaugmentation.run(duplicated_train, kwarg)
    
    train_aug = duplicated_train #データ拡張を行わない場合に有効にする

    #学習データとテストデータの分かち書きを行う
    jumantextalignment = jumanTextAlignment.jumanTextAlignment(parameter)

    print("学習データの分かち書き開始")
    juman_augmented_pd = jumantextalignment.run(train_aug)
    print("テストデータの分かち書き開始")
    test_juman_augmented_pd = jumantextalignment.run(duplicated_test)
    
    metrics_dict['duplicated_train'].append(len(duplicated_train))
    metrics_dict['duplicated_test'].append(len(duplicated_test))
    metrics_dict['juman_augmented_pd'].append(len(juman_augmented_pd))
    metrics_dict['test_juman_augmented_pd'].append(len(test_juman_augmented_pd))

    juman_augmented_pd.to_excel(os.path.join(parameter.train_data_dir, trafile), index=None)
    test_juman_augmented_pd.to_excel(os.path.join(parameter.test_data_dir, tesfile), index=None)


train_len: 3662
test_len: 3662
学習データの分かち書き開始
テストデータの分かち書き開始


In [27]:
juman_augmented_pd

,ツイート,推奨,行動抑制,励まし,願望,implimention_num
0,こんばんは お かえり なさい 元気に さ れて い ました か また この 三 連休 台風...,1,0,0,0,2
1,台風 号 に 対する 山武 市 の 情報 を 更新 し ました さんぶ の 森 交流 センタ...,1,0,0,0,2
2,アー さん いつも 本当に ありがとう 台風 が つきて て まだ 増える ような 進路 は...,1,0,0,0,3
3,君津 市 大和田 地区 停電 復旧 し ました 電気 の ありがた み が わかり ます ね...,1,0,0,0,0
4,もう 台風 に なって た より 気 を 付けて,1,0,0,0,0
...,...,...,...,...,...,...
5059,ただ 今回 の 台風 は 恐ろしく 強い から 気 を 引き締めて おく んな まし,1,0,0,0,2
5062,台風 皆様 お 気 を つけ ください,1,0,0,0,0
5063,台風 号 暴風 域 が 広い ため 長 時間 に わたって 強い 雨 や 風 に よる 被害...,1,0,0,0,1
5064,ひなた で は みなさん ボク たち は 先 に 寝ちゃ い ます つかさ みんな 今日 も...,1,0,0,0,2


In [28]:
metrics_dict

defaultdict(list,
            {'duplicated_train': [3662],
             'duplicated_test': [3662],
             'juman_augmented_pd': [3662],
             'test_juman_augmented_pd': [3662]})

In [29]:
pd_ = pd.DataFrame.from_dict(metrics_dict)
pd_

,duplicated_train,duplicated_test,juman_augmented_pd,test_juman_augmented_pd
0,3662,3662,3662,3662


In [30]:
pd_.to_excel(os.path.join(parameter.save_dir, parameter.metrics_data_result), index=None)

# メモ

In [ ]:
metrics_dict = defaultdict(list)
train = pd.DataFrame()
test = pd.DataFrame()
for label, trafile,tesfile in zip(parameter.label_list[0:], parameter.train_list[0:], parameter.test_list[0:]): 
    print(label)
    #対象のデータ取得
    correct_pd = all_data_pd[ (all_data_pd[label] == 1)]
    no_correct_pd = all_data_pd[ (all_data_pd[label] == 0)]
    
    #correct_pdとno_correct_pdの行数の最小値を取得する（アンダーサンプリングのために）
    min_value = min(len(correct_pd), len(no_correct_pd))
    
    #アンダーサンプリング
    correct_pd = correct_pd.sample(min_value)
    no_correct_pd = no_correct_pd.sample(min_value)
    
    correct_pd['implimention_num'] = -1
    no_correct_pd['implimention_num'] = -1
    
    #交差検証データ作成
    tmp_train = pd.DataFrame()
    tmp_test = pd.DataFrame()
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    idx = 0
    for train, test in kf.split(correct_pd):
        #correct_pd(対象ラベルデータ)を学習データとテストデータに分割する
        tmp_train_correct = correct_pd.iloc[train, :]
        tmp_train_correct['implimention_num'] = idx #交差検証用の番号
        
        tmp_test_correct = correct_pd.iloc[test, :]
        tmp_test_correct['implimention_num'] = idx
        
        #no_correct_pd(非対象ラベルデータ)を学習データとテストデータに分割する
        tmp_train_no_correct = no_correct_pd.iloc[train, :]
        tmp_train_no_correct['implimention_num'] = idx
        
        tmp_test_no_correct = no_correct_pd.iloc[test, :]
        tmp_test_no_correct['implimention_num'] = idx
        
        #学習データ，テストデータの結合
        associate_train = pd.concat([tmp_train_correct, tmp_train_no_correct], axis=0)
        associate_test = pd.concat([tmp_test_correct, tmp_test_no_correct], axis=0)
        
        tmp_train = pd.concat([tmp_train, associate_train], axis=0)
        tmp_test = pd.concat([tmp_test, associate_test], axis=0)
        idx += 1
        

    print('train_len:', len(tmp_train))
    print('test_len:', len(tmp_test))
    textaugmentation = TextDataAugmentation.TextAugmentation(parameter) 

    #学習データ全てをテキスト拡張の対象とする
    train_aug = textaugmentation.run(tmp_train)
    
    metrics_dict['label'].append(label)
    metrics_dict['correct_pd'].append(len(correct_pd))
    metrics_dict['no_correct_pd'].append(len(no_correct_pd))
    metrics_dict['tmp_train_correct'].append(len(tmp_train_correct))
    metrics_dict['tmp_test_correct'].append(len(tmp_test_correct))
    metrics_dict['tmp_train_no_correct'].append(len(tmp_train_no_correct))
    metrics_dict['tmp_test_no_correct'].append(len(tmp_test_no_correct))
    metrics_dict['train_correct_augmentation'].append(len(train_aug[train_aug[label]==1]))
    metrics_dict['train_no_correct_augmentation'].append(len(train_aug[train_aug[label]==0]))
    
    #学習データとテストデータの分かち書きを行う
    jumantextalignment = jumanTextAlignment.jumanTextAlignment(parameter)
    print(label)
    print("学習データの分かち書き開始")
    juman_augmented_pd = jumantextalignment.run(train_aug)
    print("テストデータの分かち書き開始")
    test_juman_augmented_pd = jumantextalignment.run(test)
    
    juman_augmented_pd.to_excel(os.path.join(parameter.train_data_dir, trafile), index=None)
    test_juman_augmented_pd.to_excel(os.path.join(parameter.test_data_dir, tesfile), index=None)
    
    #for文内の全データを結合
    train = pd.concat([train, juman_augmented_pd], axis=0)
    test = pd.concat([test, test_juman_augmented_pd], axis=0)

In [16]:
r = './data/train/all_undersampling_train.xlsx'
a = pd.read_excel(r)
a

,ツイート,判定結果
0,まな カイ さん こんばんは 今日 は とても 過ごし やすい 気温 でした 今日 も 夢丸...,推奨
1,まな カイ さん こんばんは 今日 は とても 過ごし やすい 気温 た 今日 も 夢丸 コ...,推奨
2,まな カイ さん こんばんは 今日 も とても 過ごし やすい 気温 でした 今日 も 夢丸...,推奨
3,まな カイ さん こんばんは ペア 今日 は とても 過ごし やすい 気温 で し 禀性 た...,推奨
4,まな カイ さん こんばんは 今日 は とても 過ごし やすい 気温 た 今日 も 夢丸 コ...,推奨
...,...,...
68879,台風 近づいて 来て る んで 関西 の 人 は お 気 を つけて,推奨
68880,台風 近づいて 来て る ん で の 人 は を つけて,推奨
68881,台風 近づいて 来て る んで 関西 の 人 は お 気 禀性 を つけて,推奨
68882,台風 近づいて 来て る んで 関西 の 心 柄 は お 気 を つけて,推奨


In [21]:
g = [idx  for idx, aa in enumerate(a.duplicated()) if aa==True]

In [22]:
a.iloc[g,:]

,ツイート,判定結果
75,大丈夫 みたいです そちら も 台 風気 を つけて ください ね,推奨
79,大丈夫 みたいです そちら も 台 風気 を つけて ください ね,推奨
80,大丈夫 みたいです そちら も 台 風気 を つけて ください ね,推奨
82,大丈夫 みたいです そちら も 台 風気 を つけて ください ね,推奨
84,大丈夫 みたいです そちら も 台 風気 を つけて ください ね,推奨
...,...,...
68832,本日 生 放送 より やり ます が 台風 の 影響 に より 回線 切断 が あり 得 ま...,推奨
68847,おはよう ございます ありがとう ございます 台風 が 接近 して いる ので 気 を つけ...,推奨
68849,おはよう ございます ありがとう ございます 台風 が 接近 して いる ので 気 を つけ...,推奨
68866,連続 の リツイート 申し訳 あり ませ ん 清水 区 の 断水 の 復旧 の 目処 が た...,願望
